## 1 | Import packages and set defaults

In [21]:
import os
import h5py
import numpy as np
import tkinter
from tkinter.filedialog import askopenfilename, askopenfilenames
from collections import defaultdict
print("done!")


done!


## 2 | Try to access a file

In [5]:
tkinter.Tk().withdraw() # Close the root window
in_path = askopenfilename()
print(file_path)

D:/Dropbox (UCL - SWC)/Project_paginhibition/analysis/loose_seal/analysis_sample_cells/dmpag_vgat_171123_c5_LIBD_OP_clear_VC_1.hdf5


In [12]:
folder_name = os.path.split(in_path)[0]
file_name = os.path.split(in_path)[1]
# Split the pathname path into a pair, (head, tail) where tail is the last pathname component and head is everything leading up to that. The tail part will never contain a slash; if path ends in a slash, tail will be empty. If there is no slash in path, head will be empty. If path is empty, both head and tail are empty. Trailing slashes are stripped from head unless it is the root (one or more slashes only). In all cases, join(head, tail) returns a path to the same location as path (but the strings may differ).
print(folder_name)
print(file_name)

D:/Dropbox (UCL - SWC)/Project_paginhibition/analysis/loose_seal/analysis_sample_cells
dmpag_vgat_171123_c5_LIBD_OP_clear_VC_1.hdf5


In [ ]:
def importFile():
    "What is this function doing?"
    
    tkinter.Tk().withdraw() # Close the root window
    in_path = askopenfilename()
    folder_name = os.path.split(in_path)[0]
    file_name = os.path.split(in_path)[1]
    
    voltage_mV, current_pA, command, ttl, time, dt = openFile(in_path)
    
    return voltage_mV, current_pA, command, ttl, time, dt, folder_name

In [ ]:
def openFile(in_path, curated_channel = None):
    "What is this function doing?"

    if '.tdms' in in_path:
        extracted_channels, time, dt = openTDMSfile(in_path)
    elif '.hdf5' in in_path:
        extracted_channels, time, dt = openHDF5file(in_path, curated_channel = curated_channel)
    
    voltage_mV = extracted_channels[0]
    current_pA = extracted_channels[1]
    command = extracted_channels[2]
    ttl = extracted_channels[3]
    
    return voltage_mV, current_pA, command, ttl, time, dt

In [23]:
print(in_path)
test_file = h5py.File(in_path, 'r')
list(test_file.keys())

dataDict = defaultdict(list)
list(dataDict)


D:/Dropbox (UCL - SWC)/Project_paginhibition/analysis/loose_seal/analysis_sample_cells/dmpag_vgat_171123_c5_LIBD_OP_clear_VC_1.hdf5


[]

In [ ]:
def openHDF5file(in_path,
                 channel_list = ['Channel A', 'Channel B', 'Output A', 'Output B'],
                 curated_channel = None):
    "What is this function doing?"

    hdf5_file = h5py.File(in_path, 'r')
    
    dataDict = defaultdict(list)
    
    for channel in hdf5_file.keys():
        
        #to fix hdf5 indexing
        if 'Channel' in channel or 'Output' in channel:
            trialKeysInHDF5 = list(hdf5_file[channel].keys()) # which has been sorted alphabatically as in strings. can be simulated by sorted([str(x) for x in range(numTrials)]) 
            sortI, numericalOrder = zip(*sorted(enumerate([int(x) for x in trialKeysInHDF5]),key=lambda i:i[1])) # resort numerically, and get the sorting indexes 
            
            if curated_channel is not None: 
                sortI, numericalOrder = zip(*sorted(enumerate([int(x) for x in list(hdf5_file[curated_channel].keys())]),key=lambda i:i[1]))
                if 'Channel' in channel: trialKeysInHDF5 = list(hdf5_file[curated_channel].keys())
                if 'Output' in channel: sortI = [int(x)-1 for x in numericalOrder] #note, may not work as expected if very first trial is deleted ? but doesnt matter as output is usually the same across trials in a given file
#            print(channel, trialKeysInHDF5, sortI)
        else:
            trialKeysInHDF5 = list(hdf5_file[channel].keys())
            sortI = range(len(trialKeysInHDF5))
#            print('not sorted')
            
        for i in range(len(sortI)):
            correctedTrialKey = trialKeysInHDF5[sortI[i]] 
            dataDict[channel].append(np.array(hdf5_file[channel][correctedTrialKey]))
   
    extractedChannels = []
    
    for channel in channel_list:
        extractedChannels.append(dataDict[channel])

    
    if len(dataDict['Time']) >0:
        time = dataDict['Time'][0]
        dt = np.mean(np.diff(time))
    else:
        dt = 0.04
        time = np.linspace(0, len(dataDict['Channel A'][0])*dt, len(['Channel A'][0]))
    
    return extractedChannels, time, dt

In [ ]:
def openTDMSfile(in_path, channel_list = ['Channel A', 'Channel B', 'Output A', 'Output B']):

    #returns list of arrays, each are trials i.e. [np.array(trial)]
    
    tdms_file = TdmsFile(in_path)
    
    dataDict = defaultdict(list)
    
    for group in tdms_file.groups():
        i=0
        for channel in tdms_file.group_channels(group):
            #channelname = re.findall(r"'(.*?)'", channel.path , re.DOTALL)[1]            
            dataDict[group].append(channel.data)
            i+=1
     
                       
    extractedChannels = []
    
    for channel in channel_list:
        extractedChannels.append(dataDict[channel])
    
    time = dataDict['Time'][0]
    dt = np.mean(np.diff(time))
    
    return extractedChannels, time, dt

In [ ]:
def parse_TTL_edges(TTL, edgeType):
    trial =0
    edges = []
    
    while len(edges)<1 and trial < len(TTL):
    
        if 'rising' in edgeType:
            edges = np.where(np.diff(TTL[trial])>0) 
        elif 'falling' in edgeType:
            edges = np.where(np.diff(TTL[trial])<0) 
        elif 'both' in edgeType:
            edges = np.where(np.diff(TTL[trial])!=0)    
            
        edges = edges + np.full(len(edges),1) #+1 to correct the shift due to differentiation
        edges = edges[0] #just to unwrap the array
        
        trial = trial+1
    
    
    return edges